## Annotate ClinVar entries with ACMG scores and GnomAD statistics

Example of GeneBe Python client usage for annotating variants included in the ClinGen Evidence Repository (https://erepo.clinicalgenome.org/evrepo/). This example includes parsing HGVS entities, as this is how variants are represented in the downloaded version of Repository. In the end we will compare automatic ACMG classification with the human classifications from the Repository for each of the ACMG criteria. 

This is an example of using Pandas together with GeneBe. 



In [1]:
# Remember to have required packages installed

# !pip install -U genebe
# !pip install tqdm
# !pip install ipywidgets
# !pip install scikit-learn


## Download the ClinGen Variants

In [10]:
import pandas as pd

# download the ClinGen
url = 'https://erepo.clinicalgenome.org/evrepo/api/classifications/all'

erepo = pd.read_csv(url, sep='\t')
print(erepo.shape)

host = 'https://api.genebe.net'



(6105, 20)


### Translate HGVS variants
To proceed we have to convert HGVS variants to chr-pos-ref-alt. This can easily be done using `parse_hgvs`.

Some of the variants are represented in non valid HGVS. They are omited.

In [3]:
# import genebe as gnb
import genebe as gnb


print('Version:' + gnb.__version__)
# logging.basicConfig(level=logging.INFO)

erepo_subset = erepo.dropna(subset=['#Variation']).copy()
erepo_subset = erepo_subset.reset_index()

variants_hgvs = erepo_subset['#Variation'].tolist()

variants = gnb.parse_variants(variants_hgvs,endpoint_url= f"{host}/cloud/api-public/v1/hgvs",
    batch_size=100)

erepo_subset['variant'] = variants
invalid_hgvs_rows = erepo_subset[erepo_subset['variant'] == '']

count_empty_variant = len(invalid_hgvs_rows)

# Display the count
print(f"Number of rows where 'variant' is an empty string: {count_empty_variant}")
print(invalid_hgvs_rows)



Version:0.0.14


100%|██████████| 57/57 [00:23<00:00,  2.45it/s]

Number of rows where 'variant' is an empty string: 54
      index                                         #Variation  \
143     143            NM_206933.2(USH2A):c.12295-?_14133+?del   
181     181  NM_000257.4(MYH7):c.2785_2787GAG[2] (p.Glu931del)   
913     980      NM_000277.3(PAH):c.281_283TCA[1] (p.Ile95del)   
931     999                 NM_002880.3(RAF1):c.-339_-338AG[1]   
947    1023   NM_004700.4(KCNQ4):c.803_805CCT[1] (p.Ser269del)   
1410   1546          NM_001001890.3(RUNX1):c.*2533_*2534TG[10]   
1412   1550  NM_004360.5(CDH1):c.369_375CCGCCCC[3] (p.His12...   
1413   1551      NM_004360.5(CDH1):c.436_437TC[1] (p.Pro147fs)   
1415   1553  NM_004360.5(CDH1):c.32_34TGC[7] (p.Leu14_Leu15...   
1466   1617    NM_004360.5(CDH1):c.1235_1236TA[3] (p.Ile415fs)   
1479   1630  NM_004360.5(CDH1):c.2062_2063TG[1] (p.Cys688_G...   
1485   1636       NM_004360.5(CDH1):c.32_34TGC[6] (p.Leu15dup)   
1492   1643      NM_004360.5(CDH1):c.692_693TC[2] (p.His233fs)   
1521   1693       NM_0

## Annotate variants

This single line annotate variants with ACMG and much more. Look at the list of columns. 

You can modify the `annotate_variants_list_to_dataframe` arguments to further improve the result.

In [12]:


annotations = gnb.annotate_variants_list_to_dataframe(erepo_subset['variant'].tolist(), flatten_consequences=True, endpoint_url=f"{host}/cloud/api-public/v1/variants")
annotations.columns



  0%|          | 0/57 [00:00<?, ?it/s]

100%|██████████| 57/57 [00:29<00:00,  1.95it/s]


Index(['chr', 'pos', 'ref', 'alt', 'transcript', 'gene_symbol', 'dbsnp',
       'gnomad_exomes_af', 'gnomad_exomes_ac', 'gnomad_exomes_homalt',
       'revel_score', 'bayesdelnoaf_score', 'phylop100way_score', 'acmg_score',
       'acmg_classification', 'acmg_criteria', 'gene_hgnc_id', 'hgvs_c',
       'consequences', 'gnomad_genomes_af', 'gnomad_genomes_ac',
       'gnomad_genomes_homalt', 'dbscsnv_ada_score', 'alphamissense_score',
       'apogee2_score', 'gnomad_mito_homoplasmic', 'gnomad_mito_heteroplasmic',
       'mitotip_score'],
      dtype='object')

## Concatenate with sources

Now, let's concatenate our annotations with the source database for further analysis.

We will select only columns that we need for this analysis.

In [13]:
result = pd.concat([erepo_subset, annotations], axis=1)
result = result[['#Variation','variant','Applied Evidence Codes (Met)','acmg_criteria','acmg_score']]
result.columns


Index(['#Variation', 'variant', 'Applied Evidence Codes (Met)',
       'acmg_criteria', 'acmg_score'],
      dtype='object')

## Analysis
Here, we will analyze how the Evidence Repository ACMG criteria compare with GeneBe criteria assignments.

Let's prepare the dataframe.

In [7]:
value_map = {'_Stand Alone':8,'_Stand alone':8,'_Stand_Alone':8, '_Stand_alone':8,'_Stand_Alone':8,  '_Very Strong': 8, '_Very_Strong': 8, '_Strong': 4, '_Moderate': 2, '_Supporting': 1, '': -1, 'None': 0}

criteria = ['PVS1', 'PS1', 'PS2', 'PS3', 'PS4', 'PS5', 'PM1', 'PM2', 'PM3', 'PM4', 'PM5', 'PM6', 'PM7', 'PP1', 'PP2', 'PP3', 'PP4', 'PP5', 'PP6', 'BS1', 'BS2', 'BS3', 'BS4', 'BS5', 'BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BP8', 'BA1']

default_values = {        'PS1': 4,        'PS2': 4,        'PS3': 4,        'PS4': 4,        'PS5': 4,        'PM1': 2,        'PM2': 2,        'PM3': 2,        'PM4': 2,        'PM5': 2,        'PM6': 2,        'PM7': 2,        'PP1': 1,        'PP2': 1,        'PP3': 1,        'PP4': 1,        'PP5': 1,        'PP6': 1,        'BS1': 4,        'BS2': 4,        'BS3': 4,        'BS4': 4,        'BS5': 4,        'BP1': 1,    'BP2': 1,    'BP3': 1,    'BP4': 1,    'BP5': 1,    'BP6': 1,    'BP7': 1,    'BP8': 1,    'PVS1': 8,    'BA1': 8}

# if criteria contains key-criterium, in the column set number according to custom strength
for criterium in criteria:
    key = criterium + '_erepo'
    result[key] = result['Applied Evidence Codes (Met)'].str.extract(f'{criterium}([A-Za-z _]*)', expand=False).fillna('None')
    result[key] = result[key].map(value_map)

for criterium in criteria:
    key = criterium + '_genebe'
    result[key] = result['acmg_criteria'].str.extract(f'{criterium}([A-Za-z _]*)', expand=False).fillna('None')
    result[key] = result[key].map(value_map)

# set numeric values for criteria without custom strength
for criterium in criteria:
    result[criterium+'_erepo'] = result[criterium+'_erepo'].replace(-1, default_values[criterium])
    result[criterium+'_genebe'] = result[criterium+'_genebe'].replace(-1, default_values[criterium])


for suffix in '_erepo', '_genebe':
    result['score'+suffix] = (
            result['PVS1'+suffix] +
            result['PS1'+suffix] + result['PS2'+suffix] + result['PS3'+suffix] + result['PS4'+suffix] + result['PS5'+suffix] +
            result['PM1'+suffix] + result['PM2'+suffix] + result['PM3'+suffix] + result['PM4'+suffix] + result['PM5'+suffix] +
            result['PM6'+suffix] + result['PM7'+suffix] +
            result['PP1'+suffix] + result['PP2'+suffix] + result['PP3'+suffix] + result['PP4'+suffix] + result['PP5'+suffix] +
            result['PP6'+suffix]
            - result['BS1'+suffix] - result['BS2'+suffix] - result['BS3'+suffix] - result['BS4'+suffix] - result['BS5'+suffix]
            - result['BP1'+suffix] - result['BP2'+suffix] - result['BP3'+suffix] - result['BP4'+suffix] - result['BP5'+suffix]
            - result['BP6'+suffix] - result['BP7'+suffix] - result['BP8'+suffix]
            - result['BA1'+suffix]
    )


## Correlation between scores

When considering this correlation, remember that the human judge has access to more criteria than the GeneBe algorithm.

In [8]:
import numpy as np

correlation_coefficient = np.corrcoef(result['score_erepo'], result['score_genebe'])
correlation_coefficient[0,1]




0.8296581587167354

## Score per criterium

Here, let's examine what Cohen's Kappa score looks like for each criterion when comparing ClinGen with GeneBe.

Keep in mind that a score of 0 means that this criterion is not set by GeneBe due to a lack of data.

In [9]:
from sklearn.metrics import cohen_kappa_score
import math

kappas = {}

for criterium in criteria:
    ekey = criterium + '_erepo'
    gkey = criterium + '_genebe'
    evalues = result[ekey]
    gvalues = result[gkey]
    kappa = cohen_kappa_score(evalues, gvalues)
    if not math.isnan(kappa):
        kappas[criterium] = kappa

kappas



{'PVS1': 0.8185191273065793,
 'PS1': 0.08463506882443095,
 'PS2': 0.0,
 'PS3': 0.0,
 'PS4': 0.0,
 'PM1': 0.19009215930803292,
 'PM2': 0.3822234354548125,
 'PM3': 0.0,
 'PM4': 0.5092724302972851,
 'PM5': 0.22769678423922513,
 'PM6': 0.0,
 'PP1': 0.0,
 'PP2': 0.298912072811791,
 'PP3': 0.25917740782891785,
 'PP4': 0.0,
 'PP5': 0.0,
 'BS1': 0.0918472799703457,
 'BS2': 0.20740807822023832,
 'BS3': 0.0,
 'BS4': 0.0,
 'BP1': 0.0,
 'BP2': 0.0,
 'BP3': 0.0,
 'BP4': 0.2193168963734642,
 'BP5': 0.0,
 'BP6': 0.0,
 'BP7': 0.5930384471817842,
 'BA1': 0.3726553632392755}